In [ ]:
import numpy as np
import scipy as sps
import sys
import time
import pandas as pd
import random
from matplotlib import pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Latex
from numpy import linalg
from scipy import linalg, stats
from sympy import *


In [ ]:
#this function generates the grid of different terrains and randomly picks a target

#1: flat
#2: hilly
#3: forest
#4: maze of caves

def genGrid(d):  
  grid = np.zeros((d,d)) 
  count = 0

  for i in range(d):
    for j in range(d):
      num =  np.random.randint(1,5)
      grid[i][j] = num      

  return grid

#print(genGrid(10))

In [ ]:
#this function generates a target for a grid
def genTarget(grid):
  d = len(grid)
  x = np.random.randint(0,d)
  y = np.random.randint(0,d)
  return (x,y)

In [ ]:
#this function represents the environment

def environment(terrain, target, cell):
  
  #if the cell being searched contains the target
  if cell == target:
    num = np.random.randint(1,11)
    #if the cell is flat
    if terrain == 1:
      if num > 1:
        return True
      else:
        return False
    #if the cell is hilly
    elif terrain == 2:
      if num > 3:
        return True
      else:
        return False
    #if the cell is a forest
    elif terrain == 3:
      if num > 7:
        return True
      else:
        return False
    #if the cell is a maze of caves
    elif terrain == 4:
      if num > 9:
        return True
      else:
        return False
  #if the cell does not contain target
  else:
    return False


# Basic Agent 1

In [ ]:
def basicAgent1(grid, target, x, y):
  searches = 0
  distance = 0
  found = False
  fringe = []
  d = len(grid)
  belief = {}
 

  #set initial probabilities
  for i in range(d):
    for j in range(d):
      belief[(i,j)] = 1/(d*d)

  while found != True:
    #search current cell
    check = environment(grid[x][y], target, (x,y))
    searches +=1
    #if target was found, break out of loop
    if check == True:
      found = True
      break
    else:
      #update belief for current cell by multiplying by the probability of failure given the terrain
      prob = belief[(x,y)]
      if grid[x][y] == 1:
        prob *= 0.1
      elif grid[x][y] == 2:
        prob *= 0.3
      elif grid[x][y] == 3:
        prob *= 0.7
      elif grid[x][y] == 4:
        prob *= 0.9

      belief[(x,y)] = prob
      
      #update belief for whole board
      sum = 0

      #the sum represents P(Observations)
      for key in belief:
        sum += belief[key]
      coords = []
      for i in range(d):
        for j in range(d):
          belief[(i,j)] /= sum
          #each element in coords is (x coordinate, y coordinate, probability)
          coords.append((i, j, belief[(i,j)]))
      
      #sort coords in order of highest probability 
      coords = sorted(coords, key = lambda coords: coords[2], reverse=True)

      #choose new x,y to search
      maxProb = []

      #add the first element of coords to maxProb since it has the highest probability
      maxProb.append((coords[0][0], coords[0][1]))

      #check if any other elements in coords have the same probability
      for i in range(1, len(coords)):
        if coords[i][2] == coords[0][2]:
          maxProb.append((coords[i][0], coords[i][1]))
        else:
          break
      
      #determine the closest cell to the current cell from all the coordinates with the max probability
      if len(maxProb) > 1:
        minDist = sys.maxsize
        for i in range(len(maxProb)):
          xx = maxProb[i][0]
          yy = maxProb[i][1]
          distX = abs(x - xx)
          distY = abs(y - yy)
          temp = distX + distY
          if temp < minDist:
            newX = xx
            newY = yy
            minDist = temp
      else:
        newX = maxProb[0][0]
        newY = maxProb[0][1]
      
      #add distance traveled to distance
      distX = abs(x - newX)
      distY = abs(y - newY)
      distance = distance + distX + distY

      #set x and y to new location
      x = newX
      y = newY
  
  return searches + distance



(1, 2)
[[3. 2. 4.]
 [2. 3. 4.]
 [4. 2. 1.]]
23


# Basic Agent 2

In [ ]:
def basicAgent2(grid, target, x, y):
  searches = 0
  distance = 0
  found = False
  fringe = []
  d = len(grid)
  belief = {}
 

  #set initial probabilities based on terrain type
  for i in range(d):
    for j in range(d):
      if grid[i][j] == 1:
        belief[(i,j)] = 0.9 * (1/(d*d))
      elif grid[i][j] == 2:
        belief[(i,j)] = 0.7 * (1/(d*d))
      elif grid[i][j] == 3:
        belief[(i,j)] = 0.3 * (1/(d*d))
      elif grid[i][j] == 4:
        belief[(i,j)] = 0.1 * (1/(d*d))

  while found != True:
    #search current cell
    check = environment(grid[x][y], target, (x,y))
    searches +=1

    #if the search is successful, break out of loop
    if check == True:
      found = True
      break
    else:
      #update belief for current cell by multiplying by the probability of finding the cell given the terrain
      prob = belief[(x,y)]
      if grid[x][y] == 1:
        prob *= 0.9
      elif grid[x][y] == 2:
        prob *= 0.7
      elif grid[x][y] == 3:
        prob *= 0.3
      elif grid[x][y] == 4:
        prob *= 0.1

      belief[(x,y)] = prob

      #update belief for whole board
      sum = 0

      #the sum represents P(Observations)
      for key in belief:
        sum += belief[key]
      coords = []
      for i in range(d):
        for j in range(d):
          belief[(i,j)] /= sum
          #each element in coords is (x coordinate, y coordinate, probability)
          coords.append((i, j, belief[(i,j)]))
      
      #sort coords in order of highest probability
      coords = sorted(coords, key = lambda coords: coords[2], reverse=True)
  
      #choose new x,y to search
      maxProb = []

      #add the first element of coords to maxProb since it has the highest probability
      maxProb.append((coords[0][0], coords[0][1]))

      #check if any other elements in coords have the same probability
      for i in range(1, len(coords)):
        if coords[i][2] == coords[0][2]:
          maxProb.append((coords[i][0], coords[i][1]))
        else:
          break
      
      #determine the closest cell to the current cell from all the coordinates with the max probability
      if len(maxProb) > 1:
        minDist = sys.maxsize
        for i in range(len(maxProb)):
          xx = maxProb[i][0]
          yy = maxProb[i][1]
          distX = abs(x - xx)
          distY = abs(y - yy)
          temp = distX + distY
          if temp < minDist:
            newX = xx
            newY = yy
            minDist = temp
      else:
        newX = maxProb[0][0]
        newY = maxProb[0][1]
      
      #add distance traveled to distance
      distX = abs(x - newX)
      distY = abs(y - newY)
      distance = distance + distX + distY

      #set x and y to new location
      x = newX
      y = newY
  
  return searches + distance


# Advanced Agent

In [ ]:
def advancedAgent(grid, target, x, y):
  searches = 0
  distance = 0
  found = False
  fringe = []
  d = len(grid)
  belief = {}
  search = True
  movements = [(0, 1), (1, 0), (0, -1), (-1, 0)]

  #set initial probabilities
  for i in range(d):
    for j in range(d):
      belief[(i,j)] = 1/(d*d)


  while found != True:
  
    #if the current cell is worth searching or if you are on cell of highest probability
    if search or (newX == x and newY == y):
      check = environment(grid[x][y], target, (x,y))
      searches +=1
      if check == True:
        found = True
        break
      else:
        #update belief for current cell
        prob = belief[(x,y)]
        if grid[x][y] == 1:
          prob *= 0.1
        elif grid[x][y] == 2:
          prob *= 0.3
        elif grid[x][y] == 3:
          prob *= 0.7
        elif grid[x][y] == 4:
          prob *= 0.9

        belief[(x,y)] = prob

        #update belief for whole board
        sum = 0
        for key in belief:
          sum += belief[key]
        coords = []

        for i in range(d):
          for j in range(d):
            belief[(i,j)] /= sum
            coords.append((i, j, belief[(i,j)]))

 
        coords = sorted(coords, key = lambda coords: coords[2], reverse=True)

        #choose new x,y to search
        maxProb = []
        maxProb.append((coords[0][0], coords[0][1]))
        for i in range(1, len(coords)):
          if coords[i][2] == coords[0][2]:
            maxProb.append((coords[i][0], coords[i][1]))
          else:
            break
        
        #if there are multiple cells with the highest probability, take the shoret
        if len(maxProb) > 1:
          minDist = sys.maxsize
          for i in range(len(maxProb)):
            xx = maxProb[i][0]
            yy = maxProb[i][1]
            distX = abs(x - xx)
            distY = abs(y - yy)
            temp = distX + distY
            if temp < minDist:
              newX = xx
              newY = yy
              minDist = temp
        else:
          newX = maxProb[0][0]
          newY = maxProb[0][1]
      
      #get base value of cell of highest probability, will be compared with other cells values to see if they are worth checking
      distX = abs(x - newX)
      distY = abs(y - newY)
      baseDiscount = 1/(distX+distY+1) #calculating inverse of manhattan distance to weigh the closer squares more
      baseValue = baseDiscount * belief[(newX, newY)]

     


      
    tempValue = 0
    tempX = x
    tempY = y
    #checking neighboring cells values to see if they are worth checking 
    for m in movements:
      moveX = x + m[0]
      moveY = y + m[1]
      if moveX >= d or moveY >= d or moveX < 0 or moveY < 0:
        continue
      else:
        distNewX = abs(newX - moveX)
        distNewY = abs(newY - moveY)
        discount = 1/(distNewX+distNewY+1)
        value = discount * belief[(moveX, moveY)]

        #if neighbor is higher than the base value and higher than the current maximum value
        if value >= baseValue:
          if value > tempValue:
            tempValue = value
            tempX = moveX
            tempY = moveY
            search = True

    #all neighbors have less than base value 
    if tempValue == 0:
      search = False
      if newX == tempX and newY != tempY:
        if newY > tempY:
          tempY +=1
        else:
          tempY -=1
      elif newY == tempY and newX != tempX:
        if newX > tempX:
          tempX +=1
        else:
          tempX -=1
      #goal is to the bottom right
      elif newX > tempX and newY > tempY:
        tempX += 1
      #goal is to the upper left
      elif newX < tempX and newY < tempY:
        tempX -= 1
      #goal is to upper right
      elif newX < tempX and newY > tempY:
        tempY += 1
      #goal is to bottom left
      elif newX > tempX and newY < tempY:
        tempY -= 1
    
    addX = abs(tempX - x)
    addY = abs(tempY - y)
    distance = distance + addX + addY

    x = tempX
    y = tempY
      
    
  return searches + distance



(1, 1)
[[2. 4. 4.]
 [3. 1. 4.]
 [3. 1. 1.]]
178
1
